# Parallelize a flat histogram simulation

A grand canonical ensemble transition-matrix Monte Carlo simulation is parallelized by dividing the macrostate ranges of each processor into overlapping windows.

As an exercise, compare the macrostate probability distribution output for all windows with the individual windows in the files (crit#.txt).
Attempt to unify these individual macrostate probability distributions.

In [2]:
import feasst as fst
import pyfeasst

def mc(thread, mn, mx):
    steps_per=int(1e2)
    mc = fst.MakeMonteCarlo()
    mc.set(fst.lennard_jones())
    mc.add(fst.MakeFlatHistogram(
        fst.MakeMacrostateNumParticles(
              fst.Histogram(fst.args({"width": "1", "max": str(mx), "min": str(mn)}))),
        fst.MakeTransitionMatrix(fst.args({"min_sweeps": "10"})),
        fst.args({"beta": str(1./1.5),
                  "chemical_potential": "-2.352321"})))
    mc.add(fst.MakeTrialTranslate(fst.args({"weight": "1.", "tunable_param": "1."})))
    mc.add(fst.MakeTrialTransfer(fst.args({"particle_type": "0", "weight": "4"})))
    mc.add(fst.MakeCheckEnergyAndTune(fst.args({"steps_per": str(steps_per), "tolerance": "0.0001"})))
    mc.add(fst.MakeLogAndMovie(fst.args({"steps_per": str(steps_per), "file_name": "clones" + str(thread)})))
    mc.add(fst.MakeCriteriaUpdater(fst.args({"steps_per": str(steps_per)})))
    mc.add(fst.MakeCriteriaWriter(fst.args({
        "steps_per": str(steps_per),
        "file_name": "tmp/clones" + str(thread) + "_crit.txt"})))
    return mc

windows=fst.WindowExponential(fst.args({
  "alpha": "2",
  "num": "2",
  "maximum": "50",
  "extra_overlap": "2"})).boundaries()
print(windows)

clones = fst.Clones()
for proc, win in enumerate(windows):
    clones.add(mc(proc, win[0], win[1]))
clones.initialize_and_run_until_complete()
print(clones.ln_prob().values())

((0, 35), (33, 50))
(-97.29385639838257, -92.62272462606556, -88.66432173304669, -85.0988142689932, -81.79515837641618, -78.68841499533018, -75.7691397570993, -73.0169840038016, -70.35146954859034, -67.73944163491899, -65.24765030784586, -62.857044716837144, -60.52282831545335, -58.31584409202896, -56.15988939973713, -54.0491469063282, -51.97712129530957, -50.01007864195853, -48.02062352134634, -46.09701945187986, -44.23013557109019, -42.40275378169723, -40.64067309845258, -38.90498547864179, -37.158122895654955, -35.46353420972282, -33.808286194546326, -32.112497242834074, -30.519114994491307, -28.941418277823306, -27.359172118332086, -25.803793255743148, -24.253971353785165, -22.765900318633587, -21.294270695470946, -19.863104438616652, -18.4306651172522, -17.018856297025458, -15.65681319847861, -14.34987171111168, -12.994122297378322, -11.701739110841881, -10.384454242982118, -8.982530539291325, -7.724226650943317, -6.485190818390862, -5.241876736327754, -4.001146848626433, -2.79501

Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!